In [1]:
## this is a script to draw 2-dimensional reciprocal space mapping figure using raw x-ray diffraction data
## reciprocal space mapping is an important tool to analyze XRD data, it can reveal if the thin film material is 
## strained or not compared with the substrate signal

## raw data from .dat file has format like below:

## 2theta omega intensity
##  ...   ...   ...
##  ...   ...   ...

## see the note for the meaning of parameters and the setup  

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
##%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors, ticker, cm

In [3]:
import os
cwd = os.getcwd()
print cwd

c:\users\yang\dropbox


In [4]:
#data is shared by a shared dropbox folder provided by our collaborator
df = pd.read_table("./Bi2Ir2O7 XRD Data Shared/RSM_vantec_202peak_20171009_WY85-2Bi227.dat",sep=" ",header = None)

In [5]:
df.rename(columns={list(df)[0]:'2theta'}, inplace=True)
df.rename(columns={list(df)[1]:'omega'}, inplace=True)
df.rename(columns={list(df)[2]:'intensity'}, inplace=True)

In [6]:
wavelength = 1.5406  ## K_alpha wavelength from cooper X-Ray
df['theta'] = df['2theta']*1.0/2
df.head()
#df['omega'] = -df['omega']

,2theta,omega,intensity,theta
0,48.090,-2.0,1.0,24.0450
1,48.096,-2.0,1.0,24.0480
2,48.101,-2.0,1.0,24.0505
3,48.106,-2.0,1.0,24.0530
4,48.111,-2.0,1.0,24.0555


In [7]:
##formula for qx: qx = 1/lambda*(cos(omega) - cos(2theta - omega))
##formula for qz: qz = 1/lambda*(sin(omega) + cos(2theta - omega))
df['qx_1'] = (1.0/wavelength)*(np.cos(np.radians(df['theta'] + df['omega'])) - np.cos(np.radians(df['theta']) - np.radians(df['omega'])))

#df['qx_1'] = (1.0/wavelength)*(np.cos(df['omega'].astype(float)) - np.cos(df['2theta'].astype(float) - df['omega'].astype(float)))

df['qz_1'] = (1.0/wavelength)*(np.sin(np.radians(df['theta'] + df['omega'])) + np.sin(np.radians(df['theta']) - np.radians(df['omega'])))

#df['qz_1'] = (1.0/wavelength)*(np.sin(df['omega'].astype(float)) + np.sin(df['2theta'].astype(float) - df['omega'].astype(float)))

In [8]:
## do the coordinate rotation transformation, only after tranformation we can see the Qz and Qx directly
## rotation matrix is ([cos(chi), sin(chi)],[-sin(chi), cos(chi)])

a = np.cos(np.radians(35.2643897)) # a=cos(chi)
b = np.sin(np.radians(35.2643897)) # b = sin(chi)
df['qx_new'] = df['qx_1']*a+df['qz_1']*b
df['qz_new'] = -df['qx_1']*b + df['qz_1']*a
##convert unit from A to nm
df['qx_new_nm'] = df['qx_new']*10
df['qz_new_nm'] = df['qz_new']*10
##use log(intensity) instead of intensity to have better contrast
df['intensity_log'] = np.log(df['intensity'])
df.head()
#df.to_csv("unannealed.txt",sep = '\t')

,2theta,omega,intensity,theta,qx_1,qz_1,qx_new,qz_new,qx_new_nm,qz_new_nm,intensity_log
0,48.090,-2.0,1.0,24.0450,0.018460,0.528633,0.320279,0.420969,3.202790,4.209688,0.0
1,48.096,-2.0,1.0,24.0480,0.018462,0.528695,0.320317,0.421018,3.203166,4.210182,0.0
2,48.101,-2.0,1.0,24.0505,0.018464,0.528746,0.320348,0.421059,3.203479,4.210594,0.0
3,48.106,-2.0,1.0,24.0530,0.018466,0.528798,0.320379,0.421101,3.203792,4.211005,0.0
4,48.111,-2.0,1.0,24.0555,0.018468,0.528850,0.320411,0.421142,3.204105,4.211417,0.0


In [9]:
## have a quick look of the distribution of intensity
##df.plot(x = 'qx_new_nm',y = 'intensity_log')
##df.plot(x = 'qz_new_nm',y = 'intensity_log')

In [10]:
## intensity is mainly around 3.1 (nm^-1) to 3.2(nm^-1) for qx and 4.4(nm^-1) to 4.5(nm^-1) for qz
## now let's plot the 2D mapping

In [12]:
x = df['qx_new_nm'].astype(float)
y = df['qz_new_nm'].astype(float)
z = df['intensity']*1.0/np.max(df['intensity']).astype(float)##normalize the intensity
z_log = df['intensity_log']*1.0/np.max(df['intensity_log'])##normalize the intensity

####
from matplotlib import rcParams as rc
import matplotlib.font_manager as fm
#import seaborn as sns

rc['savefig.bbox']='standard'
rc['axes.linewidth']=1
rc['axes.grid']=False
rc['grid.linewidth']=0
rc["savefig.facecolor"]='white'
rc['savefig.edgecolor']='black'
rc['figure.edgecolor']='black'    
rc["figure.facecolor"]='white'
rc["axes.facecolor"]='white'
rc['axes.edgecolor']='black'
rc['xtick.major.size'] = 5
rc['xtick.major.width'] = 1
rc['xtick.direction'] = 'in'
rc['ytick.major.size'] = 5
rc['ytick.major.width'] = 1
rc['ytick.direction'] = 'in'
rc['xtick.labelsize'] = 20
rc['ytick.labelsize'] = 20
#fm._rebuild()
#rc.update({'font.size': 5})
#rc['figure.autolayout']=False
rc["font.family"] = "Arial"
rc["image.aspect"] = 2.0
plt.locator_params(axis='y', nbins=3)
plt.locator_params(axis='x', nbins=3)
##########################
ax=plt.axes()
ax.set_xlabel('q$_x$ (1/nm)', fontsize=15)
ax.set_ylabel('q$_z$ (1/nm)', fontsize=15)

plt.xlim(2.90,3.48)
plt.ylim(4.2,4.7)
plt.xticks(size = 13)
plt.yticks(size = 13)
#plt.text(3.22, 4.5, 'YSZ(202)',
#         fontsize=25)
#plt.text(3.18, 4.35, 'Film(404)',
#         fontsize=25)
##########################
ax.set_axis_bgcolor("white")
#ax.set_facecolor('white')

#plt.scatter(x, y, z)
cm = plt.cm.get_cmap('jet')
cm.set_under(color="w")
sc = plt.scatter(x, y, c=z_log, vmin=0.15, vmax=0.4, cmap=cm,linewidths = 0)
#sc_linear = plt.scatter(x, y, c=z, vmin=0.15, vmax=0.4, cmap=cm,linewidths = 0)

plt.colorbar(sc)
plt.axes().set_aspect(0.9)
###################
#fig = plt.figure();
#ax = Axes3D(fig);
#surf = ax.plot_trisurf(x, y, z, cmap=cm.jet, linewidth=0, vmin=0.0049, vmax=0.005)
#fig.colorbar(surf, shrink=0.5, aspect=5)
################################
plt.savefig('unannealed_new.jpg',dpi=300)

plt.axis('on')
plt.show()

C:\Users\Yang\AppData\Local\Enthought\Canopy\User\lib\site-packages\ipykernel\__main__.py:50: MatplotlibDeprecationWarning: The set_axis_bgcolor function was deprecated in version 2.0. Use set_facecolor instead.


In [ ]:
## the red point is the signal from substrate since it's thick and the thin film is very thin
## the circle region below is from thin film
## our film is not on the relaxed line so it's compressed on the in-plane direction (qx direction) and expaned on  
## out-plane direction (qz direction)